<a href="https://colab.research.google.com/github/Coulomb-f/Coulomb-f/blob/master/tutorials/tools/NeMo_Forced_Aligner_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
BRANCH = 'main'

"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""

'\nYou can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.\n\nInstructions for setting up Colab are as follows:\n1. Open a new Python 3 notebook.\n2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)\n3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)\n4. Run this cell to set up dependencies.\n'

In [ ]:
import os
# either provide a path to local NeMo repository with NeMo already installed or git clone

# option #1: local path to NeMo repo with NeMo already installed
NEMO_DIR_PATH = "NeMo"

# option #2: download NeMo repo
if 'google.colab' in str(get_ipython()) or not os.path.exists(NEMO_DIR_PATH):
  !git clone -b $BRANCH https://github.com/NVIDIA/NeMo
  %cd NeMo
  !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
  %cd -

Cloning into 'NeMo'...
remote: Enumerating objects: 205709, done.
remote: Counting objects: 100% (1907/1907), done.
remote: Compressing objects: 100% (766/766), done.
remote: Total 205709 (delta 1655), reused 1170 (delta 1141), pack-reused 203802 (from 6)
Receiving objects: 100% (205709/205709), 275.02 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (154487/154487), done.
Updating files: 100% (3515/3515), done.
/content/NeMo
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-wzi4vpe0/nemo-toolkit_73e883eab19043ed97dddb56f1fb2cc1
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-

This tutorial contains external links. Each user is responsible for checking the content and the applicable licenses and determining if suitable for the intended use.

In this tutorial, we will use [NeMo Forced Aligner](https://github.com/NVIDIA/NeMo/tree/main/tools/nemo_forced_aligner) to generate token and word alignments for a video of Neil Armstrong's first steps on the moon. We will use the ASS-format subtitle files generated by NFA to add subtitles with token-by-token and word-by-word highlighting to the video.


We will use the video at this [link](https://www.nasa.gov/wp-content/uploads/static/history/alsj/a11/a11.v1092338.mov), which is in the public domain and was obtained from the NASA website [here](https://history.nasa.gov/alsj/a11/video11.html#Step). The transcript for the video is obtained from the transcript of the mission [here](https://history.nasa.gov/alsj/a11/a11transcript_tec.pdf). As referenced on this [page](https://history.nasa.gov/alsj/a11/a11trans.html), this is a raw transcript with no copyright asserted.

The alignment process is shown below. To better understand the 'Viterbi decoding' process, you can refer to our tutorial [here](https://nvidia.github.io/NeMo/blogs/2023/2023-08-forced-alignment/).



![NFA forced alignment pipeline](https://github.com/NVIDIA/NeMo/releases/download/v1.20.0/nfa_forced_alignment_pipeline.png)



We will do the following:

1. Download the source video.
2. Prepare a manifest to input to NFA.
3. Generate alignments using NFA.
4. Generate a video with subtitles with token-by-token or word-by-word highlighting.

In [ ]:
import json
import os

from IPython.display import HTML
from base64 import b64encode

# 1. Video download
We will download the video from the provided link and convert it to a higher-resolution format - the video footage will not get any clearer, but it will allow the subtitles to look sharper when we burn them in at the end of the tutorial.

In [ ]:
# first make a directory WORK_DIR that we will save all our new files in
WORK_DIR="WORK_DIR"
!mkdir $WORK_DIR
!wget https://www.nasa.gov/wp-content/uploads/static/history/alsj/a11/a11.v1092338.mov --directory-prefix=$WORK_DIR

In [ ]:
# scale up the number of pixels in video so that the subtitles will look sharp when we burn them in later
!/usr/bin/ffmpeg -loglevel warning -y -i $WORK_DIR/a11.v1092338.mov -vf "scale=-1:480" $WORK_DIR/one_small_step.mp4
# also save the audio as a separate file, so that we can use this as input to NFA
!/usr/bin/ffmpeg -loglevel warning -y -i $WORK_DIR/a11.v1092338.mov $WORK_DIR/one_small_step.wav

A note on FFMPEG commands:

*In this tutorial, instead of just calling the `ffmpeg` command, we will specifically call `/usr/bin/ffmpeg`. If we just call `ffmpeg`, there is a chance the conda version of `ffmpeg` will be called (depending on how you installed the dependencies for this tutorial). We have observed that the conda version of `ffmpeg` may generate incorrectly formatted MP4 files with the commands used, and may not be able to generate the subtitle videos which we will create at the end of this tutorial.*


In [ ]:
# display video so we know what we will be working with
mp4 = open(f'{WORK_DIR}/one_small_step.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The audio and video quality of the video isn't great - it was taken on the Moon in 1969 after all. Using NFA, we will still be able to obtain good token & word alignments despite the poor audio quality.

# 2. Prepare a manifest to input to NFA.
NFA requires a manifest as input, in the form shown in the diagram below. As we are only aligning a single audio file, our manifest will only contain one line.

**Note**: the text field is optional, but if you omit it, you need to specify `align_using_pred_text=true` in the config you feed into NFA.


![NFA usage pipeline](https://github.com/NVIDIA/NeMo/releases/download/v1.20.0/nfa_run.png)

We prepare the text obtained from [here](https://history.nasa.gov/alsj/a11/a11transcript_tec.pdf). We add `|` to the text and we will specify in the NFA config that `additional_segment_grouping_separator="|"`. This will make sure NFA produces timestamps for the groups of text that is in between the `|` characters. It also means that, by default, the ASS subtitle files will display those groups of text together. By choosing the location of `|`, you can make sure your subtitles break up the text in the desired location.

**Extra info**: Alternatively, you can specify in the NFA config that `ass_file_config.resegment_text_to_fill_space=true`, which will cause automatic grouping of words in the ASS files. You can also specify `ass_file_config.max_lines_per_segment` to set the maximum number of lines of text that will appear at a time.

In [ ]:
text = """
I'm at the foot of the ladder. The LM footpads are only depressed in the |
surface about 1 or 2 inches, although the surface appears to be very, very |
fine grained, as you get close to it. It's almost like a powder. |
Down there, it's very fine. I'm going to step off the LM now. That's one |
small step for man, one giant leap for mankind.
"""

In [ ]:
manifest_filepath = f"{WORK_DIR}/manifest.json"
manifest_data = {
    "audio_filepath": f"{WORK_DIR}/one_small_step.wav",
    "text": text
}
with open(manifest_filepath, 'w') as f:
  line = json.dumps(manifest_data)
  f.write(line + "\n")

In [ ]:
!cat $manifest_filepath

# 3. Run NFA

We run NFA by calling the `align.py` function and passing in config parameters. In our case these want to set:

**Compulsory parameters**

* `pretrained_name` (either this or a `model_path` is required) - this is the name of the ASR model that we will use to do alignment. You can use any of NeMo's CTC models for alignment.
* `manifest_filepath` - the path to the manifest specifying the audio that you want to align and its reference text.
* `output_dir` - the path to the directory where your output files will be saved.

**Optional parameters**
* `additional_segment_grouping_separator` - a string (of any length) that you will use to define splits between different segments. In our case we used "|".
* `ass_file_config.vertical_alignment` - by default this is set to `"center"`, meaning the subtitles will appear in the center of the screen. We want them to appear at the bottom of our video, so we set this parameter to `"bottom"`.
* `ass_file_config.text_already_spoken_rgb`, `ass_file_config.text_being_spoken_rgb`, `ass_file_config.text_not_yet_spoken_rgb` - these parameters define the RGB values of the colors of the text. The default colors do not show up very clearly against our video, so we set them to some different values, which will generate the colors teal, yellow, and a very pale blue.

In [ ]:
!python $NEMO_DIR_PATH/tools/nemo_forced_aligner/align.py \
  pretrained_name="stt_en_fastconformer_hybrid_large_pc" \
  manifest_filepath=$manifest_filepath \
  output_dir=$WORK_DIR/nfa_output/ \
  additional_segment_grouping_separator="|" \
  ass_file_config.vertical_alignment="bottom" \
  ass_file_config.text_already_spoken_rgb=[66,245,212] \
  ass_file_config.text_being_spoken_rgb=[242,222,44] \
  ass_file_config.text_not_yet_spoken_rgb=[223,242,239]

The alignment process should have finished successfully, let's look at some of the output files.

In [ ]:
!head $WORK_DIR/nfa_output/ctm/*/*.ctm



The token timestamps are produced directly from the Viterbi alignment process, and the word and segment timestamps are obtained from the token timestamps using a simple grouping process shown below:

![How NFA generates word and segment alignments from token alignments](https://github.com/NVIDIA/NeMo/releases/download/v1.20.0/nfa_word_segment_alignments.png)

# 4. Make subtitled video

To make the subtitled video, we will use the ASS-format files that NFA produces.

In [ ]:
!head -n20 $WORK_DIR/nfa_output/ass/words/*.ass

In [ ]:
!head -n20 $WORK_DIR/nfa_output/ass/tokens/*.ass

We burn the subtitles into the video using the below commands. We generate 2 videos, one with token-by-token highlighting, and one with word-by-word highlighting.

In [ ]:
!/usr/bin/ffmpeg -loglevel warning -y -i "$WORK_DIR/one_small_step.mp4" \
  -vf "ass=$WORK_DIR/nfa_output/ass/tokens/one_small_step.ass" \
  $WORK_DIR/one_small_step_tokens_aligned.mp4

!/usr/bin/ffmpeg -loglevel warning -y -i "$WORK_DIR/one_small_step.mp4" \
  -vf "ass=$WORK_DIR/nfa_output/ass/words/one_small_step.ass" \
  $WORK_DIR/one_small_step_words_aligned.mp4

Let's look at the resulting videos below.

You can see that the token timestamps (in the first video) are very accurate, even despite the poor audio quality of the video.

The word timestamps (in the second video) are also very good. The only noticeable mistakes are when the word has punctuation at the end (or beginning). This is because punctuation that is not separated from a word by a space is considered to be part of that word. If the alignment for the punctuation is in a region of non-speech, then the word alignment will also contain that region of non-speech.

In [ ]:
mp4 = open(f'{WORK_DIR}/one_small_step_tokens_aligned.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
mp4 = open(f'{WORK_DIR}/one_small_step_words_aligned.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)